In [4]:
import pandas as pd
import json

In [6]:
us_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

In [8]:
# Create a empty Dataframe to concat to
temp_df = pd.DataFrame({})

# Loop through all 50 US states
for state in us_states:

    # Convert states to file names
    state_filename = f"{state.replace(' ', '_')}.json"

    # Use filename to open and store file in data_list
    with open(f'./state_data/{state_filename}', 'r') as file:
        data_list = json.load(file)

    # Loop through data_list and convert each json data in dataframe
    for item in data_list:
        for data in item.values():
            data = pd.json_normalize(data)
            data['state'] = state
            temp_df = pd.concat([temp_df, data], ignore_index=True)
temp_df


FileNotFoundError: [Errno 2] No such file or directory: './state_data/Alabama.json'

In [16]:
temp_df.to_csv('../data/state_weather_data.csv', index=False)

In [17]:
temp_df['month_day'] = temp_df['date'].apply(lambda x: x[5:])

all_dates = []

for date in temp_df['month_day'].value_counts().keys():
    all_dates.append(date)

# Create a function to map dates to quarters
def assign_quarter(date):
    month = int(date.split('-')[0])  # Extract month
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

# Apply the function to assign quarters
temp_df['quarter'] = temp_df['month_day'].apply(assign_quarter)

temp_df['quarter'].value_counts()

quarter
Q2    15756
Q3    13728
Q4    13728
Q1    12324
Name: count, dtype: int64

In [18]:
test_df = temp_df.copy()

test_df['year'] = test_df['date'].apply(lambda x: x[:4])

test_df['year']

0        1980
1        1980
2        1980
3        1980
4        1980
         ... 
55531    2024
55532    2024
55533    2024
55534    2024
55535    2024
Name: year, Length: 55536, dtype: object

In [19]:
# Get rid of all object columns except the columns we use to group (state, quarter)
test_df = test_df.drop(columns=['lat', 'lon', 'tz', 'date', 'units', 'month_day'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55536 entries, 0 to 55535
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cloud_cover.afternoon  55536 non-null  float64
 1   humidity.afternoon     55536 non-null  float64
 2   precipitation.total    55536 non-null  float64
 3   temperature.min        55536 non-null  float64
 4   temperature.max        55536 non-null  float64
 5   temperature.afternoon  55536 non-null  float64
 6   temperature.night      55536 non-null  float64
 7   temperature.evening    55536 non-null  float64
 8   temperature.morning    55536 non-null  float64
 9   pressure.afternoon     55536 non-null  float64
 10  wind.max.speed         55536 non-null  float64
 11  wind.max.direction     55536 non-null  float64
 12  state                  55536 non-null  object 
 13  quarter                55536 non-null  object 
 14  year                   55536 non-null  object 
dtypes:

In [20]:
grouped_df = test_df.groupby(['state', 'quarter', 'year']).mean().reset_index()

grouped_df

,state,quarter,year,cloud_cover.afternoon,humidity.afternoon,precipitation.total,temperature.min,temperature.max,temperature.afternoon,temperature.night,temperature.evening,temperature.morning,pressure.afternoon,wind.max.speed,wind.max.direction
0,Alabama,Q1,1980,75.735000,67.498333,7.910000,277.080000,289.098333,286.361667,283.133333,282.753333,279.715000,1018.783333,5.151667,242.193333
1,Alabama,Q1,1981,47.528000,48.048000,0.092000,275.494000,287.446000,285.976000,278.474000,284.816000,276.208000,1021.624000,4.818000,221.754000
2,Alabama,Q1,1982,75.294000,73.532000,6.532000,278.410000,290.300000,288.494000,284.152000,284.844000,284.428000,1017.372000,4.676000,222.838000
3,Alabama,Q1,1983,72.450000,59.328000,2.584000,276.706000,289.176000,286.708000,279.208000,284.578000,277.054000,1013.456000,3.550000,186.388000
4,Alabama,Q1,1984,33.130000,52.585000,0.225000,275.708333,287.823333,286.011667,278.455000,284.608333,276.465000,1022.005000,3.146667,260.798333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8895,Wyoming,Q4,2019,32.666667,62.666667,0.000000,258.565000,274.773333,270.495000,261.483333,270.858333,261.608333,1022.500000,8.015000,248.833333
8896,Wyoming,Q4,2020,55.833333,49.333333,0.000000,266.928333,277.758333,274.880000,269.603333,273.068333,267.986667,1018.833333,8.458333,210.666667
8897,Wyoming,Q4,2021,61.666667,55.000000,0.608333,267.926667,279.068333,275.465000,269.971667,273.445000,268.423333,1014.666667,8.198333,228.333333
8898,Wyoming,Q4,2022,81.666667,48.500000,0.930000,270.050000,278.068333,276.268333,272.858333,274.853333,272.008333,1012.000000,10.681667,224.666667


In [21]:
grouped_df.columns

Index(['state', 'quarter', 'year', 'cloud_cover.afternoon',
       'humidity.afternoon', 'precipitation.total', 'temperature.min',
       'temperature.max', 'temperature.afternoon', 'temperature.night',
       'temperature.evening', 'temperature.morning', 'pressure.afternoon',
       'wind.max.speed', 'wind.max.direction'],
      dtype='object')

In [22]:
pivoted_df = grouped_df.pivot_table(
    index=['state', 'year'],      # We keep 'state' and 'year' as the index
    columns='quarter',            # Pivot on the 'quarter' column
    values=[                      # Specify all columns to pivot (excluding 'state', 'year', 'quarter')
        'cloud_cover.afternoon', 'humidity.afternoon', 'precipitation.total',
        'temperature.min', 'temperature.max', 'temperature.afternoon',
        'temperature.night', 'temperature.evening', 'temperature.morning',
        'pressure.afternoon', 'wind.max.speed', 'wind.max.direction'
    ]
)

# Flatten the multi-level column index
pivoted_df.columns = [f'{col[1]}_{col[0]}' for col in pivoted_df.columns]

# Reset index to turn 'state' and 'year' back into columns
pivoted_df.reset_index(inplace=True)

# Display the result
pivoted_df

,state,year,Q1_cloud_cover.afternoon,Q2_cloud_cover.afternoon,Q3_cloud_cover.afternoon,Q4_cloud_cover.afternoon,Q1_humidity.afternoon,Q2_humidity.afternoon,Q3_humidity.afternoon,Q4_humidity.afternoon,...,Q3_temperature.night,Q4_temperature.night,Q1_wind.max.direction,Q2_wind.max.direction,Q3_wind.max.direction,Q4_wind.max.direction,Q1_wind.max.speed,Q2_wind.max.speed,Q3_wind.max.speed,Q4_wind.max.speed
0,Alabama,1980,75.735,44.480000,40.675000,52.151667,67.498333,55.990000,51.900000,50.010000,...,297.231667,280.063333,242.193333,198.683333,188.051667,185.386667,5.151667,3.636667,3.155000,4.641667
1,Alabama,1981,47.528,65.207143,48.771667,35.768333,48.048000,45.842857,58.491667,64.810000,...,296.306667,283.935000,221.754000,173.021429,183.100000,99.535000,4.818000,3.534286,2.866667,3.656667
2,Alabama,1982,75.294,52.621429,56.115000,71.518333,73.532000,61.824286,65.210000,69.565000,...,295.173333,287.206667,222.838000,185.530000,131.945000,193.718333,4.676000,3.221429,3.026667,5.816667
3,Alabama,1983,72.450,53.022857,41.711667,49.433333,59.328000,57.222857,59.598333,64.446667,...,295.193333,282.078333,186.388000,235.011429,151.050000,231.078333,3.550000,4.607143,3.500000,4.715000
4,Alabama,1984,33.130,61.528333,52.196667,58.903333,52.585000,54.216667,67.193333,60.461667,...,295.041667,284.205000,260.798333,127.723333,127.528333,162.966667,3.146667,4.155000,2.831667,3.945000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Wyoming,2020,47.500,52.333333,15.500000,55.833333,62.666667,41.000000,34.666667,49.333333,...,288.616667,269.603333,216.333333,196.833333,246.333333,210.666667,8.925000,9.600000,7.565000,8.458333
2246,Wyoming,2021,55.600,42.571429,29.666667,61.666667,58.000000,35.142857,28.666667,55.000000,...,290.438333,269.971667,290.200000,285.000000,194.833333,228.333333,8.606000,7.435714,8.991667,8.198333
2247,Wyoming,2022,66.600,56.714286,43.833333,81.666667,38.000000,47.714286,24.000000,48.500000,...,291.791667,272.858333,195.600000,215.714286,171.000000,224.666667,8.458000,9.850000,8.345000,10.681667
2248,Wyoming,2023,35.800,37.571429,15.166667,81.833333,75.800000,41.142857,25.333333,64.333333,...,291.413333,268.733333,253.800000,209.000000,174.833333,176.666667,5.598000,8.510000,9.295000,5.623333


In [23]:
# Create a new feature 'is_future' to mark rows with missing (future) data
pivoted_df['is_future'] = pivoted_df.apply(lambda row: 1 if row.isnull().any() else 0, axis=1)

# Replace NaN values with -1 to indicate missing data
pivoted_df.fillna(-1, inplace=True)

# Now we have two ways of representing future data:
# 1. -1 in the weather-related columns
# 2. The 'is_future' column marking rows with future data
pivoted_df

,state,year,Q1_cloud_cover.afternoon,Q2_cloud_cover.afternoon,Q3_cloud_cover.afternoon,Q4_cloud_cover.afternoon,Q1_humidity.afternoon,Q2_humidity.afternoon,Q3_humidity.afternoon,Q4_humidity.afternoon,...,Q4_temperature.night,Q1_wind.max.direction,Q2_wind.max.direction,Q3_wind.max.direction,Q4_wind.max.direction,Q1_wind.max.speed,Q2_wind.max.speed,Q3_wind.max.speed,Q4_wind.max.speed,is_future
0,Alabama,1980,75.735,44.480000,40.675000,52.151667,67.498333,55.990000,51.900000,50.010000,...,280.063333,242.193333,198.683333,188.051667,185.386667,5.151667,3.636667,3.155000,4.641667,0
1,Alabama,1981,47.528,65.207143,48.771667,35.768333,48.048000,45.842857,58.491667,64.810000,...,283.935000,221.754000,173.021429,183.100000,99.535000,4.818000,3.534286,2.866667,3.656667,0
2,Alabama,1982,75.294,52.621429,56.115000,71.518333,73.532000,61.824286,65.210000,69.565000,...,287.206667,222.838000,185.530000,131.945000,193.718333,4.676000,3.221429,3.026667,5.816667,0
3,Alabama,1983,72.450,53.022857,41.711667,49.433333,59.328000,57.222857,59.598333,64.446667,...,282.078333,186.388000,235.011429,151.050000,231.078333,3.550000,4.607143,3.500000,4.715000,0
4,Alabama,1984,33.130,61.528333,52.196667,58.903333,52.585000,54.216667,67.193333,60.461667,...,284.205000,260.798333,127.723333,127.528333,162.966667,3.146667,4.155000,2.831667,3.945000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Wyoming,2020,47.500,52.333333,15.500000,55.833333,62.666667,41.000000,34.666667,49.333333,...,269.603333,216.333333,196.833333,246.333333,210.666667,8.925000,9.600000,7.565000,8.458333,0
2246,Wyoming,2021,55.600,42.571429,29.666667,61.666667,58.000000,35.142857,28.666667,55.000000,...,269.971667,290.200000,285.000000,194.833333,228.333333,8.606000,7.435714,8.991667,8.198333,0
2247,Wyoming,2022,66.600,56.714286,43.833333,81.666667,38.000000,47.714286,24.000000,48.500000,...,272.858333,195.600000,215.714286,171.000000,224.666667,8.458000,9.850000,8.345000,10.681667,0
2248,Wyoming,2023,35.800,37.571429,15.166667,81.833333,75.800000,41.142857,25.333333,64.333333,...,268.733333,253.800000,209.000000,174.833333,176.666667,5.598000,8.510000,9.295000,5.623333,0


In [24]:
pivoted_df.to_csv('../data/weather_data.csv', index=False)